In [12]:
import pandas as pd

#import data

url = 'https://raw.githubusercontent.com/CarsenEssing/Essing-Carsen-ml-assignment-1/refs/heads/main/Data/morg-2014-emp.csv?token=GHSAT0AAAAAAC7FDH3SPP3DVLG6HKXAYEBKZ6APWPQ'

df = pd.read_csv(url, low_memory=False)

print(df.head())
print(df.info())

   Unnamed: 0             hhid intmonth stfips     weight  earnwke  uhours  \
0           3    2600310997690  January     AL  3151.6801  1692.00      40   
1           5   75680310997590  January     AL  3457.1138   450.00      40   
2           6   75680310997590  January     AL  3936.9110  1090.00      60   
3          10  179140131100930  January     AL  3288.3640   769.23      40   
4          11  179140131100930  January     AL  3422.8500   826.92      40   

   grade92  race  ethnic  ...  ownchild  chldpres            prcitshp  state  \
0       43     1     NaN  ...         0         0  Native, Born In US     63   
1       41     2     NaN  ...         2         6  Native, Born In US     63   
2       41     2     NaN  ...         2         6  Native, Born In US     63   
3       40     1     NaN  ...         2         4  Native, Born In US     63   
4       43     1     NaN  ...         2         4  Native, Born In US     63   

                                               ind

In [14]:
#filter the dataset by computer programmers
occupation_code = 1010
df_filtered = df[df['occ2012'] == occupation_code]


#print("Number of rows", occupation_code, ":", len(df_filtered))
#print(df_filtered.head())

Number of rows for occupation code 1010 : 516
      Unnamed: 0             hhid intmonth stfips     weight  earnwke  uhours  \
539         1228    3905360695986  January     CA  3296.9904  1596.15      40   
637         1431   93943700055666  January     CA  3820.0149  2500.00      40   
1378        3096  833092919600838  January     CA  3643.5517   120.00       6   
1570        3551   56560780790091  January     CO  1619.5856  2692.30      50   
2031        4452  709019963039490  January     CT  1456.0355  1923.07      60   

      grade92  race  ethnic  ...  ownchild  chldpres            prcitshp  \
539        43     2     NaN  ...         0         0  Native, Born In US   
637        43     2     NaN  ...         0         0  Native, Born In US   
1378       44     1     1.0  ...         0         0  Native, Born In US   
1570       43     1     NaN  ...         0         0  Native, Born In US   
2031       44     1     NaN  ...         2         8  Native, Born In US   

      stat

In [16]:
#print(df_filtered.describe())

          Unnamed: 0          hhid       weight      earnwke      uhours  \
count     516.000000  5.160000e+02   516.000000   516.000000  516.000000   
mean   160722.468992  4.572279e+14  2551.036335  1411.085465   40.831395   
std     90930.452229  3.233499e+14  1248.722391   628.475023    6.504715   
min      1228.000000  5.643330e+10   193.715700     2.000000    5.000000   
25%     82180.750000  1.576628e+14  1535.997225   941.250000   40.000000   
50%    158565.500000  4.244584e+14  2822.593000  1346.150000   40.000000   
75%    238489.250000  7.490751e+14  3412.941950  1846.150000   40.000000   
max    316857.000000  9.997203e+14  6535.735200  2884.610000   80.000000   

          grade92        race     ethnic         age         sex     marital  \
count  516.000000  516.000000  22.000000  516.000000  516.000000  516.000000   
mean    42.441860    1.627907   3.318182   42.339147    1.207364    2.812016   
std      1.506785    1.240543   2.749656   11.283948    0.405812    2.61094

# Model 1: Earnings per hour and age
### Predicts an individual's earnings per hour using only their age as a predictor.
* Age is often associated with work experience, skill development, and career progression, all of which can influence earnings
* Each additional year of age is linked to a specific change in earnings per hour (regression coefficient)

In [33]:
import statsmodels.api as sm

#calculate hourly earnings
if 'earnings_per_hour' not in df.columns:
    df['earnings_per_hour'] = df['earnwke'] / df['uhours']

# Define age as the predictor
X = df_filtered[['age']]
y = df_filtered['earnings_per_hour']

X = sm.add_constant(X)

#fit
model1 = sm.OLS(y, X).fit()

print(model1.summary())


                            OLS Regression Results                            
Dep. Variable:      earnings_per_hour   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     40.28
Date:                Thu, 27 Feb 2025   Prob (F-statistic):           4.85e-10
Time:                        16:43:04   Log-Likelihood:                -2100.4
No. Observations:                 516   AIC:                             4205.
Df Residuals:                     514   BIC:                             4213.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         19.5200      2.430      8.032      0.0

# Model 2: Earnings per hour and age, education level, and gender
### explains more of the variation in hourly earnings than the baseline model that used age alone. Including these predictors acknowledges that factors beyond age, such as educational attainment and gender, can have an impact on wages.
* Grade92: Represents the highest educational grade completed,key indicator of an individual's education level which can influence earnings
* Sex: Captures differences in earnings that may be associated with gender

In [38]:
#OLS with more predictors
X1 = df_filtered[['age', 'grade92', 'sex']]
y = df_filtered['earnings_per_hour']

X1 = sm.add_constant(X1)

#fit
model2_option1 = sm.OLS(y, X1).fit()

print(model2_option1.summary())

Option 1: Predictors = age, grade92, sex
                            OLS Regression Results                            
Dep. Variable:      earnings_per_hour   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     27.88
Date:                Thu, 27 Feb 2025   Prob (F-statistic):           1.03e-16
Time:                        17:04:54   Log-Likelihood:                -2080.8
No. Observations:                 516   AIC:                             4170.
Df Residuals:                     512   BIC:                             4187.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      

# Model 3: Interaction term between age and aducation
## Adding an interaction term between age and education which captures the possibility that the effect of education on earnings per hour may change with age
* Multiplying age and grade92: tests whether the effect of education on earnings changes as individuals get older. See if higher education has larger wage benefits for older workers compared to younger workers, or vice versa.

In [47]:
df_filtered = df_filtered.copy()
df_filtered['age_grade92'] = df_filtered['age'] * df_filtered['grade92']

#interaction term: age * grade92
df_filtered['age_grade92'] = df_filtered['age'] * df_filtered['grade92']

#define predictors: age, grade92, sex, and the interaction term
X3_option1 = df_filtered[['age', 'grade92', 'sex', 'age_grade92']]
y = df_filtered['earnings_per_hour']


X3_option1 = sm.add_constant(X3_option1)

#fit 
model3_option1 = sm.OLS(y, X3_option1).fit()


print(model3_option1.summary())


                            OLS Regression Results                            
Dep. Variable:      earnings_per_hour   R-squared:                       0.147
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     22.09
Date:                Fri, 28 Feb 2025   Prob (F-statistic):           7.75e-17
Time:                        13:45:59   Log-Likelihood:                -2078.7
No. Observations:                 516   AIC:                             4167.
Df Residuals:                     511   BIC:                             4189.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const        -177.9294     58.667     -3.033      

# Model 4: More interaction terms
## includes predictors: age, education, gender, and marital status and 2 new interaction terms.

* sex_grade92: Captures whether the effect of education on earnings per hour differs by gender. The returns to education might be higher for one gender compared to the other.
* age_marital: Captures whether the effect of age on earnings per hour varies by marital status. Marital status might impact work-life balance or stability, which can interact with how age influences earnings.


In [45]:
df_filtered = df_filtered.copy()

#create interaction terms:
df_filtered['sex_grade92'] = df_filtered['sex'] * df_filtered['grade92']
df_filtered['age_marital'] = df_filtered['age'] * df_filtered['marital']

#define predictors for option 2
X4_option2 = df_filtered[['age', 'grade92', 'sex', 'marital', 'sex_grade92', 'age_marital']]
y = df_filtered['earnings_per_hour']

#add constant for the intercept
X4_option2 = sm.add_constant(X4_option2)

#fit
model4_option2 = sm.OLS(y, X4_option2).fit()

print(model4_option2.summary())


Model 4 Option 2: Model with interactions between sex and grade92, and between age and marital
                            OLS Regression Results                            
Dep. Variable:      earnings_per_hour   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.157
Method:                 Least Squares   F-statistic:                     16.98
Date:                Fri, 28 Feb 2025   Prob (F-statistic):           6.51e-18
Time:                        13:35:08   Log-Likelihood:                -2072.8
No. Observations:                 516   AIC:                             4160.
Df Residuals:                     509   BIC:                             4189.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------